# Telephone CSV 2

In [5]:
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_core.output_parsers import StrOutputParser
import pandas as pd

llm = ChatOpenAI(model="gpt-4o-mini")

In [2]:
telephone2_df = pd.read_csv("data/TelephoneDepartment_2.csv")
telephone2_df.head()

,Building,CodeName,Description,Telephone
0,NaN,ถามเบอร์,ถามเบอร์โทรศัพท์ใดๆ,1000
1,ER,USER,Counter US at ER,0338
2,ER,CTER,Tech CT at ER,"0350, 47115"
3,ER,Xray ER,Radiology Interpretation Room,"0346, 47968"
4,ER,Team ER,Ask Team ER,1182


In [9]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = f"""You are an assistant for searching hospital telephone number. Use the following <phone_book> to answer human request. 

<phone_book>
{telephone2_df.to_markdown()}
</phone_book>

If you don't know the answer, just say that you don't know. Return the output using provided <output-template>, replacing `CodeName` and `Telephone` variable with actual value.

<output-template>
**CodeName** -> `Telephone`
</output-template>
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

prompt.invoke({"input": "H"}).to_messages()

[SystemMessage(content="You are an assistant for searching hospital telephone number. Use the following <phone_book> to answer human request. \n\n<phone_book>\n|    | Building   | CodeName         | Description                   | Telephone          |\n|---:|:-----------|:-----------------|:------------------------------|:-------------------|\n|  0 | nan        | ถามเบอร์          | ถามเบอร์โทรศัพท์ใดๆ              | 1000               |\n|  1 | ER         | USER             | Counter US at ER              | 0338               |\n|  2 | ER         | CTER             | Tech CT at ER                 | 0350, 47115        |\n|  3 | ER         | Xray ER          | Radiology Interpretation Room | 0346, 47968        |\n|  4 | ER         | Team ER          | Ask Team ER                   | 1182               |\n|  5 | ER         | ER1              | ER Team 1                     | 0301               |\n|  6 | ER         | ER2              | ER Team 2                     | 0303               |\

In [10]:
chain = prompt | llm | StrOutputParser()

In [8]:
res1 = chain.invoke("Resusitation 1")
print(res1)

**`Resus1`** -> `1107`


In [11]:
res2 = chain.invoke("CT1")
print(res2)

**CT1** -> `6712`


In [12]:
res3 = chain.invoke("CT1, 2")
print(res3)

**CT1** -> `6712`  
**CT2** -> `6733`
